# 1. 导入模块和库

In [1]:
import time
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix, classification_report

E:\Anaconda\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. 导入数据+PCA

In [2]:
data = pd.read_csv(r"F:\Code_Data\2_Rocks_Spectrum_Reflectance_essay_30_origin_modify_2.csv",encoding='utf-8') #encoding='GBK',防止中文乱码
X = data.iloc[:,1:-1]
y = data.iloc[:,-1]
X_SG = savgol_filter(X, 5, 2)
X_SG_mms = MinMaxScaler().fit_transform(X_SG)
Label = LabelEncoder().fit_transform(y)
data.iloc[:,1:-1] = X_SG_mms
data.iloc[:, -1] = Label
X = data.iloc[:,1:-1]
y = data.iloc[:,-1]
X.shape, y.shape

X_dr = PCA(29).fit_transform(X)

# 3. 遗传算法

In [3]:
def optuna_objective(trial):

    #定义参数空间
    n_estimators = trial.suggest_int("n_estimators",50,150,1) #整数型，(参数名称，下界，上界，步长)
    max_depth = trial.suggest_int("max_depth",30,50,1)
    max_features = trial.suggest_int("max_features",5,30,1)
    min_samples_leaf = trial.suggest_int("min_samples_leaf",1,51,1)
    min_samples_split = trial.suggest_int("min_samples_split",2,22,1)

    #定义评估器
    #需要优化的参数由上述参数空间决定
    #不需要优化的参数则直接填写具体值
    rfc = RFC(n_estimators = n_estimators
              ,max_depth = max_depth
              ,max_features = max_features
              ,min_samples_leaf = min_samples_leaf
              ,min_samples_split = min_samples_split
              ,random_state=0
              ,verbose=False
              ,n_jobs=-1
             )

    #交叉验证过程，输出准确率(accuracy)
    cv = KFold(n_splits=5,shuffle=True,random_state=0)
    score = cross_val_score(rfc,X_dr,y
                            ,scoring='accuracy'
                            ,cv=cv #交叉验证模式
                            ,verbose=False #是否打印进程
                            ,n_jobs=-1 #线程数
                            ,error_score='raise' #如果在某个折叠中出现错误，会引发一个异常
                           )
    #最终输出平均准确率
    return np.mean(score)

In [4]:
def optimizer_optuna(n_trials, algo):

    #定义使用随机网格搜索
    sampler = optuna.samplers.NSGAIISampler(population_size=50  # int，种群大小，即每代中包含的个体数量。默认 50
                                           ,mutation_prob=1.0 # float，变异概率，指定了在交叉之后每个个体发生变异的概率。默认 1.0。
                                           ,crossover_prob=1.0  # float，交叉概率，指定了在每次交叉操作中，一个个体与另一个个体产生子代的概率。默认 1.0
                                           #,crossover_operator  # str，交叉操作符的类型。可选 "sbx"（模拟二进制交叉）和 "ux"（单点均匀交叉）。默认"sbx"
                                           #,mutation_operator  # str，变异操作符的类型。可选 "polynomial"（多项式变异）和 "random"（随机变异）。默认"polynomial"
                                           #,eta= # float，多项式变异操作的参数，用于控制变异的强度。较高的值表示更强的变异。默认20.0。
                                           #,crossover_eta= # float，模拟二进制交叉操作的参数，用于控制交叉的强度。较低的值表示更强的交叉。默认20.0。
                                           #,max_generations=  # int，最大的迭代代数。默认值为 None，表示没有最大代数限制。
                                           )

    #实际优化过程，首先实例化优化器
    study = optuna.create_study(sampler = sampler #要使用的具体算法
                                ,direction="maximize" #优化的方向，可以填写minimize或maximize
                                #,storage="sqlite:///db.sqlite3" # Specify the storage URL here.
                                #,study_name="quadratic-simple"
                               )
    #开始优化，n_trials为允许的最大迭代次数
    #由于参数空间已经在目标函数中定义好，因此不需要输入参数空间
    study.optimize(optuna_objective #目标函数
                   ,n_trials=n_trials #最大迭代次数（包括最初的观测值的）
                   ,show_progress_bar=True #要不要展示进度条呀？
                  )

    #可直接从优化好的对象study中调用优化的结果
    #打印最佳参数与最佳损失值
    print("\n","\n","best params: ", study.best_trial.params,
          "\n","\n","best score: ", study.best_trial.values,
          "\n")

    return study.best_trial.params, study.best_trial.values

In [5]:
def optimized_optuna_search_and_report(n_trials, algo):
    start_time = time.time()

    # 进行搜索优化
    best_params, best_score = optimizer_optuna(n_trials, algo)

    # 打印最佳参数和分数
    print("\n","\n","best params: ", best_params,
          "\n","\n","best score: ", best_score,
          "\n")

    end_time = time.time()
    time_random = (end_time - start_time) / 60  # 转换为分钟
    print(f"Optimization completed in {time_random} minutes.")

    return best_params, best_score, time_random

# 执行优化
best_params, best_score, time_random = optimized_optuna_search_and_report(30,optuna.samplers.NSGAIISampler())

[I 2023-09-20 09:15:19,752] A new study created in memory with name: no-name-ea8a0c3c-11d6-4f6f-bc27-e989a19898cc
Best trial: 0. Best value: 0.866335:   3%|█▌                                            | 1/30 [00:58<28:25, 58.79s/it]

[I 2023-09-20 09:16:18,567] Trial 0 finished with value: 0.8663347743490857 and parameters: {'n_estimators': 113, 'max_depth': 43, 'max_features': 12, 'min_samples_leaf': 21, 'min_samples_split': 7}. Best is trial 0 with value: 0.8663347743490857.


Best trial: 0. Best value: 0.866335:   7%|███                                           | 2/30 [02:17<32:53, 70.48s/it]

[I 2023-09-20 09:17:37,229] Trial 1 finished with value: 0.8161150120005217 and parameters: {'n_estimators': 104, 'max_depth': 30, 'max_features': 24, 'min_samples_leaf': 45, 'min_samples_split': 21}. Best is trial 0 with value: 0.8663347743490857.


Best trial: 2. Best value: 0.912327:  10%|████▌                                         | 3/30 [03:59<38:08, 84.74s/it]

...

[I 2023-09-20 09:48:23,115] Trial 27 finished with value: 0.8446914445125537 and parameters: {'n_estimators': 117, 'max_depth': 39, 'max_features': 5, 'min_samples_leaf': 28, 'min_samples_split': 11}. Best is trial 2 with value: 0.9123267430244175.


Best trial: 2. Best value: 0.912327:  97%|███████████████████████████████████████████▌ | 29/30 [34:05<00:48, 48.84s/it]

[I 2023-09-20 09:49:25,658] Trial 28 finished with value: 0.810788853364882 and parameters: {'n_estimators': 106, 'max_depth': 40, 'max_features': 20, 'min_samples_leaf': 48, 'min_samples_split': 4}. Best is trial 2 with value: 0.9123267430244175.


Best trial: 2. Best value: 0.912327: 100%|█████████████████████████████████████████████| 30/30 [34:39<00:00, 69.30s/it]

[I 2023-09-20 09:49:58,849] Trial 29 finished with value: 0.8317559006288882 and parameters: {'n_estimators': 82, 'max_depth': 45, 'max_features': 13, 'min_samples_leaf': 38, 'min_samples_split': 16}. Best is trial 2 with value: 0.9123267430244175.

 
 best params:  {'n_estimators': 111, 'max_depth': 33, 'max_features': 20, 'min_samples_leaf': 2, 'min_samples_split': 22} 
 
 best score:  [0.9123267430244175] 


 
 best params:  {'n_estimators': 111, 'max_depth': 33, 'max_features': 20, 'min_samples_leaf': 2, 'min_samples_split': 22} 
 
 best score:  [0.9123267430244175] 

Optimization completed in 34.651785385608676 minutes.


# 4. 贝叶斯优化

In [20]:
def optuna_objective(trial):

    #定义参数空间
    n_estimators = trial.suggest_int("n_estimators",50,150,1) #整数型，(参数名称，下界，上界，步长)
    max_depth = trial.suggest_int("max_depth",25,50,1)
    max_features = trial.suggest_int("max_features",5,30,1)
    min_samples_leaf = trial.suggest_int("min_samples_leaf",1,51,1)
    min_samples_split = trial.suggest_int("min_samples_split",2,22,1)

    #定义评估器
    #需要优化的参数由上述参数空间决定
    #不需要优化的参数则直接填写具体值
    rfc = RFC(n_estimators = n_estimators
              ,max_depth = max_depth
              ,max_features = max_features
              ,min_samples_leaf = min_samples_leaf
              ,min_samples_split = min_samples_split
              ,random_state=0
              ,verbose=False
              ,n_jobs=-1
             )

    #交叉验证过程，输出准确率(accuracy)
    cv = KFold(n_splits=5,shuffle=True,random_state=0)
    score = cross_val_score(rfc,X_dr,y
                            ,scoring='accuracy'
                            ,cv=cv #交叉验证模式
                            ,verbose=False #是否打印进程
                            ,n_jobs=-1 #线程数
                            ,error_score='raise' #如果在某个折叠中出现错误，会引发一个异常
                           )
    #最终输出平均准确率
    return np.mean(score)

In [21]:
def optimizer_optuna(n_trials, algo):

    #定义使用TPE或者GP
    if algo == "TPE":
        algo = optuna.samplers.TPESampler(n_startup_trials = 15, n_ei_candidates = 12)
    elif algo == "GP":
        from optuna.integration import SkoptSampler
        import skopt
        algo = SkoptSampler(skopt_kwargs={'base_estimator':'GP', #选择高斯过程
                                          'n_initial_points':30, #初始观测点10个
                                          'acq_func':'EI'} #选择的采集函数为EI，期望增量
                           )

    #实际优化过程，首先实例化优化器
    study = optuna.create_study(sampler = algo #要使用的具体算法
                                ,direction="maximize" #优化的方向，可以填写minimize或maximize
                               )
    #开始优化，n_trials为允许的最大迭代次数
    #由于参数空间已经在目标函数中定义好，因此不需要输入参数空间
    study.optimize(optuna_objective #目标函数
                   ,n_trials=n_trials #最大迭代次数（包括最初的观测值的）
                   ,show_progress_bar=True #要不要展示进度条呀？
                  )

    #可直接从优化好的对象study中调用优化的结果
    #打印最佳参数与最佳损失值
    print("\n","\n","best params: ", study.best_trial.params,
          "\n","\n","best score: ", study.best_trial.values,
          "\n")

    return study.best_trial.params, study.best_trial.values

In [22]:
def optimized_optuna_search_and_report(n_trials, algo):
    start_time = time.time()

    # 进行贝叶斯优化
    best_params, best_score = optimizer_optuna(n_trials, algo)

    # 打印最佳参数和分数
    print("\n","\n","best params: ", best_params,
          "\n","\n","best score: ", best_score,
          "\n")

    end_time = time.time()
    time_bayes = (end_time - start_time) / 60  # 转换为分钟
    print(f"Optimization completed in {time_bayes} minutes.")

    return best_params, best_score, time_bayes

# 执行优化
best_params, best_score, time_bayes = optimized_optuna_search_and_report(30, "TPE")

[I 2023-08-30 10:01:18,374] A new study created in memory with name: no-name-c3763327-ac2a-4515-9eea-debb9deded2e
Best trial: 0. Best value: 0.956714:   3%|█▍                                         | 1/30 [02:52<1:23:18, 172.35s/it]

[I 2023-08-30 10:04:10,723] Trial 0 finished with value: 0.9567135547815333 and parameters: {'n_estimators': 111, 'max_depth': 44, 'max_features': 30, 'min_samples_leaf': 2, 'min_samples_split': 9}. Best is trial 0 with value: 0.9567135547815333.


Best trial: 1. Best value: 0.96094:   7%|███▏                                           | 2/30 [03:23<41:45, 89.47s/it]

[I 2023-08-30 10:04:42,179] Trial 1 finished with value: 0.960940240439346 and parameters: {'n_estimators': 144, 'max_depth': 38, 'max_features': 6, 'min_samples_leaf': 16, 'min_samples_split': 20}. Best is trial 1 with value: 0.960940240439346.


Best trial: 1. Best value: 0.96094:  10%|████▋                                          | 3/30 [03:44<26:03, 57.91s/it]

...

[I 2023-08-30 10:32:02,799] Trial 27 finished with value: 0.963561090072718 and parameters: {'n_estimators': 145, 'max_depth': 33, 'max_features': 8, 'min_samples_leaf': 1, 'min_samples_split': 10}. Best is trial 23 with value: 0.9654213049919669.


Best trial: 23. Best value: 0.965421:  97%|██████████████████████████████████████████▌ | 29/30 [32:00<01:12, 72.90s/it]

[I 2023-08-30 10:33:18,959] Trial 28 finished with value: 0.9590804544292917 and parameters: {'n_estimators': 150, 'max_depth': 32, 'max_features': 12, 'min_samples_leaf': 12, 'min_samples_split': 6}. Best is trial 23 with value: 0.9654213049919669.


Best trial: 23. Best value: 0.965421: 100%|████████████████████████████████████████████| 30/30 [32:48<00:00, 65.63s/it]

[I 2023-08-30 10:34:07,139] Trial 29 finished with value: 0.962630964741877 and parameters: {'n_estimators': 139, 'max_depth': 36, 'max_features': 8, 'min_samples_leaf': 5, 'min_samples_split': 9}. Best is trial 23 with value: 0.9654213049919669.

 
 best params:  {'n_estimators': 129, 'max_depth': 33, 'max_features': 9, 'min_samples_leaf': 1, 'min_samples_split': 5} 
 
 best score:  [0.9654213049919669] 


 
 best params:  {'n_estimators': 129, 'max_depth': 33, 'max_features': 9, 'min_samples_leaf': 1, 'min_samples_split': 5} 
 
 best score:  [0.9654213049919669] 

Optimization completed in 32.812969688574476 minutes.


# 5. 最佳参数效果

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(X_dr, y, test_size=0.3, random_state=0)

In [13]:
# 默认参数
rfc_3 = RFC(n_estimators=1000,random_state=0)
rfc_3 = rfc_3.fit(X_train, Y_train)
pred_3 = rfc_3.predict(X_test)

# 遗传算法搜索评价指标
accuracy_score_3_train = rfc_3.score(X_train, Y_train)
accuracy_score_3_test = rfc_3.score(X_test, Y_test)
cm_3 = confusion_matrix(Y_test, pred_3)

In [14]:
accuracy_score_3_train, accuracy_score_3_test

(1.0, 0.9664694280078896)

In [ ]:
# 遗传算法搜索最佳参数
rfc_1 = RFC(n_estimators=97
            ,max_depth=47
            ,max_features=12
            ,min_samples_leaf=9
            ,min_samples_split=19
            ,random_state=0
           )
rfc_1 = rfc_1.fit(X_train, Y_train)
pred_1 = rfc_1.predict(X_test)

# 遗传算法搜索评价指标
accuracy_score_1_train = rfc_1.score(X_train, Y_train)
accuracy_score_1_test = rfc_1.score(X_test, Y_test)
kappa_score_1 = cohen_kappa_score(Y_test, pred_1)
cm_1 = confusion_matrix(Y_test, pred_1)

In [21]:
accuracy_score_1_train, accuracy_score_1_test

(0.9661794902766034, 0.9602704987320372)

In [10]:
# 贝叶斯优化最佳参数
rfc_2 = RFC(n_estimators=129
            ,max_depth=33
            ,max_features=9
            ,min_samples_leaf=1
            ,min_samples_split=5
            ,random_state=0
           )
rfc_2 = rfc_2.fit(X_train, Y_train)
pred_2 = rfc_2.predict(X_test)

# 贝叶斯优化评价指标
accuracy_score_2_train = rfc_2.score(X_train, Y_train)
accuracy_score_2_test = rfc_2.score(X_test, Y_test)
kappa_score_2 = cohen_kappa_score(Y_test, pred_2)
cm_2 = confusion_matrix(Y_test, pred_2)
report = classification_report(Y_test, pred_2, output_dict=True)

In [11]:
accuracy_score_2_train, accuracy_score_2_test

(0.9955308612151226, 0.9642152719075796)

In [12]:
report = classification_report(Y_test, pred_2,output_dict=True)
# 获取加权平均的精确率、召回率、F1系数
weighted_precision = report['macro avg']['precision']
weighted_recall = report['macro avg']['recall']
weighted_f1 = report['macro avg']['f1-score']

print(f"macro Precision: {weighted_precision}")
print(f"macro Recall: {weighted_recall}")
print(f"macro F1 Score: {weighted_f1}")

macro Precision: 0.9616981229160063
macro Recall: 0.9619139535985805
macro F1 Score: 0.9615055485452823


In [13]:
class_accuracy = {label: metrics['precision'] for label, metrics in report.items() if label != 'accuracy'}
df = pd.DataFrame(list(class_accuracy.items()), columns=['Class', 'Accuracy'])
df

,Class,Accuracy
0,0,0.991667
1,1,1.000000
2,2,0.990654
3,3,1.000000
4,4,0.991525
5,5,0.991525
6,6,0.975410
7,7,0.872881
8,8,0.873786
9,9,0.956522


In [14]:
cm_2_ = confusion_matrix(Y_test, pred_2, normalize='pred') # 计算列归一化的混淆矩阵
cm_2_normalized = (np.round(cm_2_ * 100)).astype(int)

In [15]:
# 保存混淆矩阵
output_folder = 'D:/1_科研进展/小论文/小论文图片/混淆矩阵'

csv_filename_2 = 'final_RF.csv'
np.savetxt(f'{output_folder}/{csv_filename_2}', cm_2, delimiter=',', fmt='%d')

csv_filename_3 = 'final_RF_Normalized.csv'
np.savetxt(f'{output_folder}/{csv_filename_3}', cm_2_normalized, delimiter=',', fmt='%d')